<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Testing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D, \
                             Dropout, Input, MaxPool2D, Rescaling, BatchNormalization
from tensorflow.keras.models import Sequential, Model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

![img](https://live.staticflickr.com/65535/51304224388_f5f4136338_h.jpg)

### Notes

- Check grad-cam, CAMs, Saliency Maps
- Implement transferlearning
  - Which backbone to use? Why was it chosen?
  - [Mehendale N. et al.](https://pubmed.ncbi.nlm.nih.gov/34075355/) use different models for all the subcases of the whole problem
    - They use VGG16, DenseNet and ResNet. Here is a [diagram](https://www.ncbi.nlm.nih.gov/pmc/articles/instance/8152712/bin/42979_2021_695_Fig1_HTML.jpg)
    - Check out [this](https://www.analyticsvidhya.com/blog/2022/03/x-ray-classification-using-pretrained-stacked-model/) article also
- Model explainability
  - Check out [this](https://neptune.ai/blog/explainability-auditability-ml-definitions-techniques-tools) post
  - [IBMs](https://www.ibm.com/topics/explainable-ai) thingy

### Code

In [52]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

use `with_info` to get dict with the features of the dataset you dumb-dumb

In [38]:
ds_info.features

FeaturesDict({
    'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
    'id': Text(shape=(), dtype=string),
    'image': Image(shape=(32, 32, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
})

### Dataset to numpy:

In [5]:
# Create a subset of ds_train and ds_val
ds_train_subset = ds_train.take(500)
ds_val = ds_train.skip(600).take(50)

# Convert datasets to np arrays
def dataset_to_numpy(dataset):
    features = []
    labels = []

    for feature, label in tfds.as_numpy(dataset):
        features.append(feature)
        labels.append(label)

    return np.array(features), np.array(labels)

# Convert ds_train_subset and ds_val to np arrays
np_train_features, np_train_labels = dataset_to_numpy(ds_train_subset)
np_val_features, np_val_labels = dataset_to_numpy(ds_val)

In [6]:
print("-------TRAIN--------")
print(np_train_features.shape)
print(np_train_labels.shape)
print("-------VALIDATION--------")
print(np_val_features.shape)
print(np_val_labels.shape)

-------TRAIN--------
(500, 16, 32, 32, 3)
(500, 16)
-------VALIDATION--------
(50, 16, 32, 32, 3)
(50, 16)


In [7]:
np_test_features, np_test_labels = dataset_to_numpy(ds_test)
print("-------TEST--------")
print(np_test_features.shape)
print(np_test_labels.shape)

-------TEST--------
(625, 16, 32, 32, 3)
(625, 16)


In [8]:
CLASS_NUM = np_test_labels.shape[1]
print(CLASS_NUM)

16


### Preprocessing #2

In [53]:
def preprocess(image, label):
    image = tf.image.per_image_standardization(image)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

In [58]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
CLASS_NUM = ds_info.features['label'].num_classes

ds_train_subset = ds_train.cache().shuffle(1000).take(6000).batch(BATCH_SIZE)
ds_train_subset = ds_train_subset.prefetch(AUTO)

# ds_val = ds_train.cache().shuffle(1000).skip(6100).take(600).batch(BATCH_SIZE)
ds_val = ds_train.prefetch(AUTO).skip(6100).take(600).batch(BATCH_SIZE)

ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [59]:
tf.keras.backend.clear_session()
model = Sequential([

    Conv2D(32, (3, 3), activation='relu', input_shape=ds_info.features['image'].shape),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(CLASS_NUM, activation='softmax')

    ])

In [60]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [64]:
# Train the model
history = model.fit(ds_train_subset, epochs=100, validation_data = ds_val)

Epoch 1/100
94/94 [==============================] - 2s 20ms/step - loss: 1.0684 - accuracy: 0.6892 - val_loss: 0.9218 - val_accuracy: 0.7750
Epoch 2/100
94/94 [==============================] - 3s 26ms/step - loss: 1.0611 - accuracy: 0.6928 - val_loss: 0.8376 - val_accuracy: 0.7917
Epoch 3/100
94/94 [==============================] - 3s 28ms/step - loss: 1.0156 - accuracy: 0.7038 - val_loss: 0.9508 - val_accuracy: 0.7617
Epoch 4/100
94/94 [==============================] - 2s 19ms/step - loss: 1.0396 - accuracy: 0.6992 - val_loss: 0.8952 - val_accuracy: 0.7800
Epoch 5/100
94/94 [==============================] - 2s 20ms/step - loss: 1.0233 - accuracy: 0.7077 - val_loss: 0.8814 - val_accuracy: 0.7567
Epoch 6/100
94/94 [==============================] - 2s 25ms/step - loss: 1.0136 - accuracy: 0.7070 - val_loss: 0.8154 - val_accuracy: 0.7900
Epoch 7/100
94/94 [==============================] - 2s 20ms/step - loss: 0.9994 - accuracy: 0.7083 - val_loss: 0.8718 - val_accuracy: 0.7683
Epoch 

In [63]:
model.save('./model.h5')